In [44]:
%load_ext autoreload
%autoreload 2

import wandb
from dotenv import dotenv_values
import pandas as pd

from src.utils import download_table
from src.models.analysis import delong_roc_test

import numpy as np


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
our_model_preds = download_table("20gczthf","test_predictions")
xgb_preds = download_table("1k5awvp8","test_predictions")

In [77]:
our_model_preds["date"] = pd.to_datetime(our_model_preds["date"]).dt.date
xgb_preds["date"] = pd.to_datetime(xgb_preds["date"]).dt.date

In [78]:
xgb_preds[xgb_preds["label"]==1].sort_values("date")

,participant_id,date,label,pred
71931,fe020e5092a736a9125f7cd671cc6ad4,2020-02-16,1,0.305184
59419,eae88ae68906f5d3b0803c4be76077ff,2020-02-16,1,0.019163
87003,7a96679e5e8041233dc1e86ea2ce06cb,2020-02-17,1,0.028514
87288,eae88ae68906f5d3b0803c4be76077ff,2020-02-17,1,0.674527
25020,7f5d75d670dbfbae0d5b7dcfb22d1f83,2020-02-17,1,0.000516
...,...,...,...,...
107327,f146536684daa1525d815cd43f3bafd6,2020-03-31,1,0.000986
57192,0fcd3f13e0172dd5176f128a2eaa3b02,2020-03-31,1,0.024949
52136,6f6cd9f458f65bd30da0ac16ef83c02a,2020-03-31,1,0.002209
94871,14bbdca33aad9f88c9669aa227696d1e,2020-03-31,1,0.006315


In [79]:
our_model_preds[our_model_preds["label"]==1].sort_values("date")

,participant_id,date,label,pred
65848,fe020e5092a736a9125f7cd671cc6ad4,2020-02-16,1,0.818525
12222,eae88ae68906f5d3b0803c4be76077ff,2020-02-16,1,0.013699
34105,15c397b567534ea2decfe5e238e685d2,2020-02-17,1,0.001193
47863,dfd3bf65ce12b59a493074e7f482499e,2020-02-17,1,0.153722
79838,7f5d75d670dbfbae0d5b7dcfb22d1f83,2020-02-17,1,0.000970
...,...,...,...,...
3707,6f6cd9f458f65bd30da0ac16ef83c02a,2020-03-31,1,0.002274
33495,14bbdca33aad9f88c9669aa227696d1e,2020-03-31,1,0.009719
50782,f146536684daa1525d815cd43f3bafd6,2020-03-31,1,0.000553
43973,0fcd3f13e0172dd5176f128a2eaa3b02,2020-03-31,1,0.064280


In [80]:
index = ["participant_id","date"]
merged = our_model_preds.set_index(index).join(xgb_preds.set_index(index),lsuffix="_ours",rsuffix="_xgb").dropna()

In [81]:
log_p = delong_roc_test(merged["label_ours"].values, merged["pred_ours"].values, merged["pred_xgb"].values)[0][0]
10**log_p

0.4377603009925714

In [82]:
merged["label_ours"].value_counts()

0    114806
1       178
Name: label_ours, dtype: int64

In [36]:
n = len(merged)
log_p = delong_roc_test(merged["label_ours"].values, merged["pred_ours"].values, np.ones(n) )[0][0]
10**log_p

7.76778087508268e-53

In [51]:
from sklearn.metrics import roc_auc_score

def bootstrap_roc_auc(labels, preds):
    rocs = []
    n = len(preds)
    for i in range(1000):
        inds = np.random.choice(np.arange(n), size=n, replace=True, p=None)
        rocs.append(roc_auc_score(labels[inds],preds[inds]))
    lower = np.quantile(rocs,0.025)
    upper = np.quantile(rocs,0.975)
    mean = np.mean(rocs)
    return lower, mean, upper

bootstrap_roc_auc(merged["label_ours"].values, merged["pred_ours"].values)

(0.7770612899704024, 0.8162924030876072, 0.8529476386694037)

In [52]:
bootstrap_roc_auc(merged["label_ours"].values, merged["pred_xgb"].values)

(0.7699339869357218, 0.808395959860667, 0.8434003010845132)

In [73]:
def hier_bootstrap_roc_auc(labels, preds):
    rocs = []
    n = len(preds)
    pos = labels.nonzero()[0]
    n_pos = len(pos)

    neg = np.ones(n)
    neg[pos] = 0
    neg = np.where(neg)[0]
    
    for i in range(1000):
        pos_inds = np.random.choice(pos, size=n_pos, replace=True, p=None)
        inds = np.concatenate([neg,pos_inds])
        rocs.append(roc_auc_score(labels[inds],preds[inds]))
    lower = np.quantile(rocs,0.025)
    upper = np.quantile(rocs,0.975)
    mean = np.mean(rocs)
    return lower, mean, upper

hier_bootstrap_roc_auc(merged["label_ours"].values, merged["pred_ours"].values)

[  1016   1051   1372   1880   2220   2604   2898   3707   4085   5126
   5366   5547   5763   5832   6115   6189   6748   6843   7952   8600
   9215   9763  10565  10589  11689  11711  12222  12353  13094  13201
  13252  13921  14435  15268  15375  15965  15992  18949  20490  23817
  25772  27363  27522  28860  29178  29610  29926  30305  31564  31853
  32227  32694  32886  33009  33495  34105  34186  36338  36650  36771
  37276  38203  38326  39704  39958  42212  43201  43973  44297  46129
  47863  49114  49402  49947  50735  50745  50782  51443  51581  52169
  53732  54320  54534  54884  55294  55535  57358  58528  59092  59106
  60090  60528  61726  62302  63013  63087  63173  64807  65098  65103
  65217  65848  66950  67531  67572  68761  69502  70231  70665  71571
  71923  72400  72502  73490  73621  73682  74077  74224  74571  74871
  74940  75462  76163  77026  77141  77160  77536  78321  79838  80088
  80385  80395  80528  80725  82066  83034  83752  84175  84211  85785
  8598

(0.7798485285485021, 0.8179907785816307, 0.8545377716575906)

In [83]:
hier_bootstrap_roc_auc(merged["label_ours"].values, merged["pred_xgb"].values)

[  1016   1051   1372   1880   2220   2604   2898   3707   4085   5126
   5366   5547   5763   5832   6115   6189   6748   6843   7952   8600
   9215   9763  10565  10589  11689  11711  12222  12353  13094  13201
  13252  13921  14435  15268  15375  15965  15992  18949  20490  23817
  25772  27363  27522  28860  29178  29610  29926  30305  31564  31853
  32227  32694  32886  33009  33495  34105  34186  36338  36650  36771
  37276  38203  38326  39704  39958  42212  43201  43973  44297  46129
  47863  49114  49402  49947  50735  50745  50782  51443  51581  52169
  53732  54320  54534  54884  55294  55535  57358  58528  59092  59106
  60090  60528  61726  62302  63013  63087  63173  64807  65098  65103
  65217  65848  66950  67531  67572  68761  69502  70231  70665  71571
  71923  72400  72502  73490  73621  73682  74077  74224  74571  74871
  74940  75462  76163  77026  77141  77160  77536  78321  79838  80088
  80385  80395  80528  80725  82066  83034  83752  84175  84211  85785
  8598

(0.7712453869419579, 0.8089002624016246, 0.8460418988202277)